In [ ]:
#Importing required libraries

import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torch.utils.tensorboard import SummaryWriter

#checking if its using my gpu or cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
#Splitting the Dataset (Train & Validation Sets)

data_dir = "./Data/train"
csv_path = "./Data/trainLabels.csv"

df = pd.read_csv(csv_path)

class_names = sorted(df['label'].unique())
class_map = {name: i for i, name in enumerate(class_names)}


train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
print(f"Train samples: {len(train_df)}, Validation samples: {len(val_df)}")


In [ ]:
#preprocessing images to be compatible with the VGG16 model

train_transform = transforms.Compose([
    transforms.Resize((224, 224)), #makes the size of the image 224x224 (cuz the vgg model has been trained on it)
    
    #Brownies: Data augmentation
    
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], #model was trained on the imagenet dataset so these are the mean and std RGB values of the images.
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), #to convert the image to a pyTorch tensor (ie bring it from range 0-255 to 0-1)
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class ImageDataset(Dataset):
    def __init__(self, dataframe, data_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_id = row['id']
        label_str = row['label']

        img_path = os.path.join(self.data_dir, f"{image_id}.png")
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = class_map[label_str]
        return image, label

train_dataset = ImageDataset(train_df, data_dir, transform=train_transform)
val_dataset = ImageDataset(val_df, data_dir, transform=val_transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
base_model = models.vgg16(weights='IMAGENET1K_V1') #brownie:loading a pretrained model 

for param in base_model.features[:20].parameters():
    param.requires_grad = False #freezing of the first 20 layers in the model which handle feature extraction and don't need to be retrained as they've already been trained on a large dataset and know basic features.

in_features = base_model.classifier[6].in_features #1000 neurons for imagenet FC layer
base_model.classifier[6] = nn.Sequential(
    nn.Linear(in_features, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, len(class_names))    
)

model = base_model.to(device)

print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-3)

#brownie: learning rate scheduler.
#reduces the learning rate when a metric like val_loss starts stagnates instead of improving.
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

patience = 3
best_val_loss = float('inf')
epochs_no_improve = 0

num_epochs = 11

train_losses, val_losses = [], []
train_accs, val_accs = [], []

writer = SummaryWriter(log_dir='Logs/with_brownies')


In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    train_epoch_loss = running_loss / len(train_loader.dataset)
    train_epoch_acc = 100.0 * correct / total

    model.eval()
    val_running_loss = 0.0
    val_correct, val_total = 0, 0
    
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            
            val_running_loss += val_loss.item() * val_images.size(0)
            
            _, val_predicted = torch.max(val_outputs, 1)
            val_correct += (val_predicted == val_labels).sum().item()
            val_total += val_labels.size(0)
    
    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_acc = 100.0 * val_correct / val_total
    
    scheduler.step(val_epoch_loss)
    
    train_losses.append(train_epoch_loss)
    val_losses.append(val_epoch_loss)
    train_accs.append(train_epoch_acc)
    val_accs.append(val_epoch_acc)
    
    writer.add_scalar('Loss/Train', train_epoch_loss, epoch)
    writer.add_scalar('Loss/Val', val_epoch_loss, epoch)
    writer.add_scalar('Accuracy/Train', train_epoch_acc, epoch)
    writer.add_scalar('Accuracy/Val', val_epoch_acc, epoch)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_acc:.2f}% "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.2f}%")
    
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_brownie_model.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered!")
            break
            
writer.close()


In [ ]:
model.load_state_dict(torch.load("best_brownie_model.pth"))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for val_images, val_labels in val_loader:
        val_images, val_labels = val_images.to(device), val_labels.to(device)
        outputs = model(val_images)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(val_labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='weighted')
rec = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Validation Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec:.3f}, Recall: {rec:.3f}, F1-score: {f1:.3f}")

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))